In [2]:
def extract_feature(X,sampling_rate,chroma,mel,mfcc):
    if  chroma:
        stft=np.abs(librosa.stft(X))
    result=np.array([])
    if  mfcc:
        mfcc=np.mean(librosa.feature.mfcc(X,sr=sampling_rate,n_mfcc=40).T,axis=0)
        result=np.hstack((result,mfcc))
    if chroma:
        chroma=np.mean(librosa.feature.chroma_stft(S=stft,sr=sampling_rate).T,axis=0)
        result=np.hstack((result,chroma))
    if mel:
        mel=np.mean(librosa.feature.melspectrogram(X,sr=sampling_rate).T,axis=0)
        result=np.hstack((result,mel))
    return result

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

ModuleNotFoundError: No module named 'google'

load audio file


In [ ]:
import librosa
audio_path = '/content/drive/MyDrive/audio dataset/Actor_01/03-01-01-01-01-01-01.wav'
x,sr = librosa.load(audio_path)

listen audio file

In [ ]:
import IPython.display as ipd
ipd.Audio(audio_path)

plot the wave form

In [ ]:
%matplotlib inline
import sklearn
import matplotlib.pyplot as plt
import librosa.display
plt.figure()
librosa.display.waveplot(x,sr)

plot the spectograph

In [ ]:
X = librosa.stft(x)
Xdb = librosa.amplitude_to_db(abs(X))
plt.figure()
librosa.display.specshow(Xdb, sr=sr, x_axis='time', y_axis='hz')
plt.colorbar()

feature extraction(MFCC)


In [ ]:
mfccs = librosa.feature.mfcc(x, sr=sr)
print(mfccs.shape)

librosa.display.specshow(mfccs, sr=sr, x_axis='time')

In [ ]:
import os
import numpy as np
path = '/content/drive/My Drive/audio dataset'
a = []
for subdir, dirs, files in os.walk(path):
    for file in files:
        try:
            x,sr = librosa.load(os.path.join(subdir,file),res_type='kaiser_fast')
            mfccs = np.mean(librosa.feature.mfcc(y=x,sr=sr,n_mfcc=40).T,axis = 0)
            file = file[6:8]
            arr = mfccs,file
            a.append(arr)
        except ValueError:
            continue

In [ ]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
x, y = zip(*a)
x = np.asarray(x)
y = np.asarray(y)
y=y.astype(np.float32)
y-=1
x_train, x_test, y_train,y_test = train_test_split(x,y,test_size=0.25)
x_test.shape, y_test.shape

In [ ]:
def plot_history(history):
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='lower right')
  # function to plot accuracy vs epoch

In [ ]:
import tensorflow as tf
model=tf.keras.models.Sequential([
   tf.keras.layers.Flatten(),                                                                
   tf.keras.layers.Dense(7200,activation='sigmoid',input_shape=(40,1)),                                                                
   tf.keras.layers.Dense(1024,activation='sigmoid'),
   tf.keras.layers.Dropout(0.2),
   tf.keras.layers.Dense(256,activation='sigmoid'),
   tf.keras.layers.Dropout(0.45),                          
   tf.keras.layers.Dense(8,activation='softmax'),
   ]) 

model.compile(optimizer='rmsprop',loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
history = model.fit(x_train, y_train, epochs=100,
                   validation_data=(x_test, y_test))

plot_history(history)
model.summary()
 


In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
y_pred=model.predict(x_test)
y_pred=np.argmax(y_pred,axis=1)
labels=('neutral','calm','happy','sad','angry','fear','disgust','suprise')
cm=confusion_matrix(y_test,y_pred,normalize='true')
fig, axis = plt.subplots(figsize=(10, 10))
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=labels)
disp.plot(ax=axis)

In [ ]:
model_cnn=tf.keras.models.Sequential([
  tf.keras.layers.Conv1D(128,(3),activation='relu',input_shape=(40,1)),                                  
  tf.keras.layers.MaxPooling1D(1),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Conv1D(64,(3),activation='relu'),
  tf.keras.layers.MaxPooling1D(2),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(256,activation='relu',kernel_regularizer=tf.keras.regularizers.L2(2e-4)),
  tf.keras.layers.Dropout(0.1),
  tf.keras.layers.Dense(128,activation='relu',kernel_regularizer=tf.keras.regularizers.L2(3e-4)),
  tf.keras.layers.Dense(8,activation='softmax')
  ])
model_cnn.compile(optimizer='rmsprop',loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
# -------------------------------
print(x_train.shape)
history = model_cnn.fit(x_train, y_train, epochs=300,
                    validation_data=(x_test, y_test))

plot_history(history)
model_cnn.summary()